In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

from trott import *
from qiskit import IBMQ
from qiskit.providers.aer import QasmSimulator
from mitiq import zne
import numpy as np

# Prepare Circuits

In [3]:
trott_gate = gen_3cnot_trott_gate() # using the basic trott method (this can be changed)
trott_steps_range = np.arange(4,25,1)
qcs = gen_st_qcs_range(trott_gate, trott_steps_range, decompose=True)

# Prepare Backend

In [4]:
# load IBMQ Account data

# IBMQ.save_account(TOKEN)  # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
provider = IBMQ.load_account()

In [5]:
# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')
# properties = jakarta.properties()

# Simulated backend based on ibmq_jakarta's device noise profile
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

# Noiseless simulated backend
sim = QasmSimulator()

# Example mitiq 

In [6]:
test_circ = qcs[10][0]

In [7]:
def executor(st_circuit, shots=8192, backend = sim_noisy_jakarta):
    shots = 8192
    reps = 8

    # create jobs
    jobs = []
    for _ in range(reps):
        # execute
        job = execute(st_circuit, backend, shots=shots)
        print('Job ID', job.job_id())
        jobs.append(job)
        
    # Average counts  
    avg_counts = {}
    for job in jobs:
        counts = job.result().get_counts()
        avg_counts = add_dicts(avg_counts, counts)
        
    
    #return job
    
    # All reps should have the same Pauli string
    pauli_string = extract_key(job.result().results[0].header.name)
    
    num_qubits = 3
    parsed_data = {}
    for active_spots in ACTIVE_LIST: # Loops through all possible parity measurements, e.g. [ZXY, ZXI, ZIY, IXY, ZII, IXI, IIY]
        for readout_string, count in avg_counts.items(): # loops through all readout values, e.g. '0x6', '0x2'
            adjusted_pauli_string, parity_meas = calc_parity_full(pauli_string, readout_string, active_spots) # ("ZXY", "0x6", (1,1,0)) -> "ZXI", 1 corresponds to <ZXI> = -1 measurement
            if adjusted_pauli_string not in parsed_data:
                parsed_data[adjusted_pauli_string] = [0,0] # [counts of 1, counts of -1]
            parsed_data[adjusted_pauli_string][parity_meas] += count

    parity = {}
    for parity_string, count in parsed_data.items():
        norm = np.sum(count)
        parity[parity_string] = (1)*count[0]/norm + (-1)*count[1]/norm
        
    print("Pauli string:", pauli_string)
    #print("Calculated parities:", parity)
    return parity[pauli_string]




In [21]:
executor(qcs[5][0])

Job ID ddc86002-98f3-4486-bde5-fe3fd401b3f3
Job ID d2b56caf-1c42-468e-9177-4bd502cf3fb8
Job ID bb13fdec-3803-41a0-8b2c-01c9855cce1c
Job ID ea74e6e2-0a9c-4cd1-848d-65808f3db707
Job ID 0818f019-d9d6-4710-9fe7-2dea47afc4b3
Job ID 7770f577-5260-4aac-b201-feb102f340c4
Job ID 9574570e-6454-47b5-84e2-82ea7f9f942c
Job ID 2174a483-ddf8-42a1-932c-dcb9b5a4a32f
Pauli string: XXX


0.022308349609375

In [20]:
zne.execute_with_zne(qcs[5][0], executor)

Job ID c76cce42-73ab-47b7-9308-1905b3dc94b2
Job ID fcd24ba6-22ee-4d69-bb64-10a8c6e0fd89
Job ID df2911eb-c684-42f1-92ed-2ca83c059289
Job ID b5622155-fd1f-4e06-8fa7-6274ab0a14fc
Job ID 512a696b-e16b-4c46-a052-7cae139de08f
Job ID d3441f94-f528-44f1-94e1-5630f70e1f85
Job ID d33930c9-63f4-4268-94e0-5e828220fef8
Job ID fb16254d-ebba-4e51-b57b-b4fc8986bc6c
Pauli string: r-1
Job ID 5711a96a-66f7-4ac7-aa7a-390c684b6b2e
Job ID e7dea68e-76a2-43d5-b7c1-2522fe7c2053
Job ID 582eb313-fb76-4916-9878-56ae81cf1288
Job ID 107cc5ba-f0ce-41bc-b7bc-7e5fbf6f8f02
Job ID 0224968d-a515-4585-ae8b-cfe40d5dd123
Job ID 26eeb0f7-e4ac-4b21-978b-7dc6840cee1b
Job ID 17f59090-54d6-4c86-9d2d-34668b918d8c
Job ID 15844546-3f3e-4aaa-a5c5-287f8ffcbdeb
Pauli string: r-2
Job ID 7ba0c232-beb9-47d9-b6c4-df22edaec354
Job ID 317257c2-1580-4091-9a11-d95ddae21c34
Job ID fc43be7a-2f77-42f4-ba46-776f07e735e9
Job ID b4be22bf-c42c-4cfe-9859-dc442d5a48f5
Job ID 86507c86-a68b-4f4b-b16f-7a53d0a367a2
Job ID d5ca0d05-2403-4f7c-b89f-e8dcb17cb

0.02490234374999997

In [8]:
def mitiq_ZNE(tomography_circuits: List[QuantumCircuit]):
    
    for circuit in tomography_circuits:
        
        zne_result = zne.execute_with_zne(circuit, executor)
        unmitigated_result = executor(circuit)
        
    
    print("Unmitigated Result:", unmitigated_result)
    print("ZNE Result:", zne_result)
    
    return unmitigated_result, zne_result

In [23]:
from mitiq.zne.inference import LinearFactory, RichardsonFactory, PolyFactory
from mitiq.zne.scaling import fold_gates_at_random, fold_global, fold_gates_from_right, fold_all
from functools import partial

In [31]:
factory = RichardsonFactory(scale_factors=[1.0, 3.0, 5.0])
noise_scaling_function = partial(
    fold_gates_from_right,
    fidelities = {"single": 1.0, "CNOT": 0.98}
)

zne.execute_with_zne(qcs[14][-1], executor, factory=factory, scale_noise=noise_scaling_function)

Job ID 6b44c090-f129-47d4-86a6-e980b1f10c55
Job ID a14cc6bc-8067-43c1-9192-5e282a81f8c0
Job ID f97f6257-2d85-4072-bed1-7c43f20c8063
Job ID 93db211a-77c2-4c7d-be91-c922c3b59f53
Job ID 8cb6dd03-835a-4365-857a-bac82fea26b0
Job ID afce245d-7c38-4713-9b12-d86ea936e695
Job ID 01927884-0ad1-4418-8ffc-8a0a63086fca
Job ID 2dd0902c-7a8c-4f41-9c06-925391e57ff5
Pauli string: r-2
Job ID e7179116-dccc-4265-aa7d-75ff207174b3
Job ID 42718b8a-8eec-4c6d-9c33-a298df67951c
Job ID cf987ee0-db4b-4bc1-ba63-00a74be49346
Job ID 1e5e12a9-f98e-466b-99f7-e515dac16412
Job ID 3807cc37-3bbe-4158-8fe6-62b8d9ea15c0
Job ID 9e24f49b-d12e-46e6-9b8d-c244789f1dea
Job ID 240eab39-a6e6-4840-b03c-aed551045759
Job ID 5a885b14-d8a0-4845-a4c3-a813439ce7a0
Pauli string: r-3
Job ID f0ef781a-3f1f-42d4-a45b-922ea6272761
Job ID 4907beed-498d-434f-b233-56fc7ea3ecbc
Job ID 22461a3a-3330-4037-a202-657c5053d6d4
Job ID c3f11fad-076f-4b44-9ca1-449ef228f03f
Job ID 45c43c6f-0e0e-41c8-a3b6-990348397a71
Job ID 9140c11c-ee15-42a3-ac80-b9d9455d8

-0.2672958374023437